In [ ]:
!pip install -q transformers datasets accelerate peft bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.4 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login()

In [ ]:
# Loading the small dataset

from datasets import load_dataset

# Load only 1000 samples for quick fine-tuning
dataset = load_dataset("yelp_review_full", split="train[:1000]")

# Show example
print(dataset[0])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/299M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'label': 4, 'text': "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."}


In [ ]:
# We will use the above We’ll use this as instruction-style prompt: “Classify the sentiment of this review.”
# Preprocessing the dataset
def format_example(example):
    return {
        "text": f"### Instruction:\nClassify the sentiment of this review:\n{example['text']}\n\n### Response:\n{example['label']}"
    }

dataset = dataset.map(format_example)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# Loading the Qwen1.5-0.5B model with 4-bit quantization ( QLoRA)
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "Qwen/Qwen1.5-0.5B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  # Important for batching

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [ ]:
#Using PEFT to apply LoRA (QLoRA) via PEFT
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn", "q_proj", "v_proj"],  # Qwen uses similar naming
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


In [ ]:
# Tokenizing the data
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=256)

tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
#Configuring the training
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./qwen-qlora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=1,
    fp16=True,
    logging_steps=10,
    save_strategy="no",
    report_to="none"
)

trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
# Starting the Finetuning
trainer.train()


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,3.210200
20,3.116500
30,2.987900
40,2.979700
50,2.965500
60,2.879600
70,2.998300
80,2.881200
90,2.919400
100,2.922500


TrainOutput(global_step=125, training_loss=2.9642484970092773, metrics={'train_runtime': 107.5177, 'train_samples_per_second': 9.301, 'train_steps_per_second': 1.163, 'total_flos': 474918420480000.0, 'train_loss': 2.9642484970092773, 'epoch': 1.0})

In [ ]:
# Saving the trained model:
model.save_pretrained("qwen-qlora-yelp")
tokenizer.save_pretrained("qwen-qlora-yelp")


('qwen-qlora-yelp/tokenizer_config.json',
 'qwen-qlora-yelp/special_tokens_map.json',
 'qwen-qlora-yelp/chat_template.jinja',
 'qwen-qlora-yelp/vocab.json',
 'qwen-qlora-yelp/merges.txt',
 'qwen-qlora-yelp/added_tokens.json',
 'qwen-qlora-yelp/tokenizer.json')

In [ ]:
# Now Testing the model:
import torch

# Sample input for testing
instruction = "### Instruction:\nClassify the sentiment of this review:\nI loved the food and service!\n\n### Response:\n"

inputs = tokenizer(instruction, return_tensors="pt").to("cuda")

# Generate output
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=10,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past

### Instruction:
Classify the sentiment of this review:
I loved the food and service!

### Response:
3...     .


In [ ]:
# ["Very bad", "Bad", "Neutral", "Good", "Excellent"]   1,2,3,4,5


In [ ]:
samples = [
    "Terrible food and rude staff.",
    "The ambiance was nice, and the pasta was amazing.",
    "Worst experience ever.",
    "Everything was perfect!",
    "It was okay, not great but not bad."
]

for review in samples:
    prompt = f"### Instruction:\nClassify the sentiment of this review:\n{review}\n\n### Response:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=10)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Review: {review}\nPrediction: {result.split('### Response:')[-1].strip()}\n")


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past

Review: Terrible food and rude staff.
Prediction: 0.........



Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in 

Review: The ambiance was nice, and the pasta was amazing.
Prediction: 4.........



Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in 

Review: Worst experience ever.
Prediction: 0.........



Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in 

Review: Everything was perfect!
Prediction: 4.........



Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in 

Review: It was okay, not great but not bad.
Prediction: 3.........



In [ ]:
# Pushing to Huggingface
model.push_to_hub("BruceLee1234/qwen-qlora-yelp-review-classifier")
tokenizer.push_to_hub("BruceLee1234/qwen-qlora-yelp-review-classifier")


adapter_model.safetensors:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/BruceLee1234/qwen-qlora-yelp-review-classifier/commit/d8dbb63d8a5a623235bf6d9f0aa26cd3f5406884', commit_message='Upload tokenizer', commit_description='', oid='d8dbb63d8a5a623235bf6d9f0aa26cd3f5406884', pr_url=None, repo_url=RepoUrl('https://huggingface.co/BruceLee1234/qwen-qlora-yelp-review-classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='BruceLee1234/qwen-qlora-yelp-review-classifier'), pr_revision=None, pr_num=None)

In [ ]:
!pip uninstall -y peft
!pip install -q git+https://github.com/huggingface/peft.git


Found existing installation: peft 0.16.1.dev0
Uninstalling peft-0.16.1.dev0:
  Successfully uninstalled peft-0.16.1.dev0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

base_model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen1.5-0.5B",
    device_map="auto",
    torch_dtype="auto"
)

# Load the PEFT adapter
peft_model = PeftModel.from_pretrained(base_model, "BruceLee1234/qwen-qlora-yelp-review-classifier")

# ✅ Merge using method on the model (no import)
peft_model = peft_model.merge_and_unload()

# Save merged model locally
peft_model.save_pretrained("merged-qwen-yelp")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/836 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

In [ ]:
# Now loading the merged model

from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("merged-qwen-yelp")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B")


In [ ]:
# Tsting the merged model
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model = AutoModelForCausalLM.from_pretrained("merged-qwen-yelp", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B")

# Example prompt
prompt = "Classify this review: The food was amazing and the service was top-notch."

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=50)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Classify this review: The food was amazing and the service was top-notch.  I had the chicken and waffles and it was so good.  I had the chicken and waffles and it was so good.  I had the chicken and waffles and it was so good.  I had the chicken and waffles


In [ ]:
#🚀 Option 2: Publish the merged model to Hugging Face Hub
from transformers import AutoTokenizer

model.push_to_hub("qwen-yelp-merged")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B")
tokenizer.push_to_hub("qwen-yelp-merged")


model.safetensors:   0%|          | 0.00/1.86G [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/BruceLee1234/qwen-yelp-merged/commit/574893afc5618b468b794e4c0fd0fc9d94c8c909', commit_message='Upload tokenizer', commit_description='', oid='574893afc5618b468b794e4c0fd0fc9d94c8c909', pr_url=None, repo_url=RepoUrl('https://huggingface.co/BruceLee1234/qwen-yelp-merged', endpoint='https://huggingface.co', repo_type='model', repo_id='BruceLee1234/qwen-yelp-merged'), pr_revision=None, pr_num=None)